In [62]:

# pytorch mlp for regression
from numpy import vstack
import numpy as np
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Tanh
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
 

with open('results.csv', 'a') as f:
    f.write('layer1,layer2,actFunc,learning,mse,mae\n') 
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # print(len(self.X[0]))
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.20):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs,layer1, layer2, act_func):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, layer1)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = act_func()
        # second hidden layer
        self.hidden2 = Linear(layer1, layer2)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = act_func()
        # third hidden layer and output
        self.hidden3 = Linear(layer2, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # print(CSVDataset(path).__getitem__(0))
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model, learn):
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=learn)
    # enumerate epochs
    for epoch in range(1000):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat
 
# prepare the data
path = 'DataSet1 - Beacon1.csv'
train_dl, test_dl = prepare_data(path)
# print(len(train_dl.dataset), len(test_dl.dataset))

In [64]:
layers1 = [10,30,50,70,90]
layers2 = [10,30,50,70,90]
actFunc = [Sigmoid,Tanh,ReLU]
learning = [0.1,0.01,0.001,0.0001]
for a in layers1:
    for b in layers2:
        for c in actFunc:
            for d in learning:
                # define the network
                
                model = MLP(10,a, b, c)
                # train the model
                train_model(train_dl, model,d)
                # evaluate the model
                mse, mae = evaluate_model(test_dl, model)
                with open('results.csv', 'a') as f:
                    f.write(f'{a},{b},{c},{d},{mse},{mae}\n')
                print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
                print('MAE: %.3f' % (mae))

                # make a single prediction (expect class=1)
                row = [-56,-54,-60,-54,-58,-60,-55,-59,-59,-54]
                yhat = predict(row, model)
                print('Predicted: %.3f' % yhat)

MSE: 3.299, RMSE: 1.816
MAE: 1.591
Predicted: 2.940
MSE: 3.181, RMSE: 1.783
MAE: 1.571
Predicted: 3.256
MSE: 2.441, RMSE: 1.562
MAE: 1.278
Predicted: 3.795
MSE: 2.935, RMSE: 1.713
MAE: 1.475
Predicted: 3.773
MSE: 3.186, RMSE: 1.785
MAE: 1.566
Predicted: 3.366
MSE: 3.182, RMSE: 1.784
MAE: 1.567
Predicted: 3.334
MSE: 3.181, RMSE: 1.784
MAE: 1.571
Predicted: 3.247
MSE: 2.985, RMSE: 1.728
MAE: 1.480
Predicted: 4.086
MSE: 3.187, RMSE: 1.785
MAE: 1.573
Predicted: 3.204
MSE: 2.267, RMSE: 1.506
MAE: 1.352
Predicted: 4.300
MSE: 1.987, RMSE: 1.410
MAE: 1.179
Predicted: 3.702
MSE: 2.298, RMSE: 1.516
MAE: 1.292
Predicted: 3.850
MSE: 3.201, RMSE: 1.789
MAE: 1.563
Predicted: 3.432
MSE: 3.180, RMSE: 1.783
MAE: 1.569
Predicted: 3.300
MSE: 3.182, RMSE: 1.784
MAE: 1.571
Predicted: 3.250
MSE: 3.071, RMSE: 1.752
MAE: 1.543
Predicted: 3.446
MSE: 3.290, RMSE: 1.814
MAE: 1.589
Predicted: 2.953
MSE: 3.186, RMSE: 1.785
MAE: 1.573
Predicted: 3.209
MSE: 3.182, RMSE: 1.784
MAE: 1.571
Predicted: 3.244
MSE: 2.797, 